In [1]:
import pandas as pd
import numpy as np

from unidecode import unidecode
import redditcleaner
import re

from collections import defaultdict

In [64]:
def clean_text(s):
    """ Basic preprocessing: encode characters, remove markdown, and remove links """
    s = unidecode(s)
    s = redditcleaner.clean(s)
    s = re.sub(r'https?://\S+', '', s)
    s = s.replace('&amp;', 'and')
    
    return s

def process_data(df):
    print('Number of rows:', len(df))
    
    df = df[df['selftext'] != '']
    df = df[df['selftext'] != 'NaN']
    df = df[df['selftext'] != '[deleted]']
    df = df[df['selftext'] != '[removed]']
    if ('stickied' in df):
        df = df[df['stickied'] == False]
    if 'distinguished' in df:
        df = df[df.distinguished != 'moderator']

    df = df[['title', 'selftext']]
    print('Number of posts:', len(df))
    
    df.selftext = df.selftext.apply(clean_text)
    df = df[df.selftext != '']
    df = df.reset_index(drop=True)
    print('Number of cleaned posts:', len(df))
    
    df['text'] = df.title + ' ' + df.selftext
    df = df.drop(['title', 'selftext'], axis=1)
    
    return df

In [65]:
df = pd.read_pickle('relationships_raw.pkl')
df = process_data(df)
df.to_pickle('relationships_text.pkl')

Number of rows: 445504
Number of posts: 93034
Number of cleaned posts: 93029


In [76]:
df = pd.read_pickle('wsb_raw.pkl')
df = process_data(df)
df.to_pickle('wsb_text.pkl')

Number of rows: 504752
Number of posts: 46648
Number of cleaned posts: 45860
